# Chronos-2 SFT+LoRA — Evaluation Dump (window × ticker)

This notebook **does not train** any model.

It loads the checkpoints produced by `chronos2_sft_lora.ipynb` and runs an evaluation procedure consistent with `core/eval.py::evaluate_model_on_test`, **but additionally saves per-window, per-ticker results** to Parquet files.

These dumps are then used by the companion notebook `chronos2_sft_lora_stats_tests.ipynb` to run paired statistical tests (ticker-level).


In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

# Project imports
current_dir = os.getcwd()
REPO_ROOT = os.path.dirname(current_dir) # Using this notation to keep the rest of the code the same.
sys.path.append(REPO_ROOT)


from tiingo_data.download_data import get_daily_returns_data_cached
from core.data import prepare_data_for_chronos
from core.eval import evaluate_model_on_test
from utils import get_device
from chronos import Chronos2Pipeline

DEVICE = get_device()
print("REPO_ROOT:", REPO_ROOT)
print("DEVICE:", DEVICE, "| CUDA:", torch.cuda.is_available())


REPO_ROOT: c:\Users\rosar\chronos_dnlp
DEVICE: cuda | CUDA: True


In [2]:
# Load data and evaluation split

df_all = get_daily_returns_data_cached()
df_train_clean, df_eval_clean = prepare_data_for_chronos(df_all, test_size=1200)

print("Train shape:", df_train_clean.shape)
print("Eval  shape:", df_eval_clean.shape)
print("Eval date range:", df_eval_clean.index.min(), "→", df_eval_clean.index.max())


Train shape: (2797, 114)
Eval  shape: (1200, 114)
Eval date range: 2021-02-12 00:00:00+00:00 → 2025-11-20 00:00:00+00:00


In [3]:
# Load best hyperparameters (tuning results)

CSV_CANDIDATES = [
    Path(REPO_ROOT) / "notebooks" / "outputs" / "tuning_results" / "tuning_best_results.csv",
    Path(REPO_ROOT) / "outputs" / "tuning_results" / "tuning_best_results.csv"
]
best_csv_path = next((p for p in CSV_CANDIDATES if p.exists()), None)
if best_csv_path is None:
    raise FileNotFoundError(
        "tuning_best_results.csv not found. Checked:\n" + "\n".join(map(str, CSV_CANDIDATES))
    )

df_best = pd.read_csv(best_csv_path)
df_best["group"] = df_best["group"].astype(str).str.strip()
hp_by_group = df_best.set_index("group").to_dict(orient="index")

PREDICTION_LENGTH = 1

def get_hparams(group: str) -> dict:
    g = str(group).strip()
    if g not in hp_by_group:
        raise KeyError(f"Group '{g}' not found in tuning CSV. Available: {sorted(hp_by_group.keys())}")
    row = hp_by_group[g]
    return dict(
        prediction_length=PREDICTION_LENGTH,
        context_length=int(row["context_length"]),
        num_steps=int(row["num_steps"]),
        batch_size=int(row["batch_size"]),
        learning_rate=float(row["learning_rate"]),
        stride=int(row.get("stride", 50)),
        n_eval_samples=int(row.get("n_eval_samples", 100)),
    )

global_hp = get_hparams("global")
print("[HP global]", global_hp)


[HP global] {'prediction_length': 1, 'context_length': 128, 'num_steps': 1500, 'batch_size': 48, 'learning_rate': 3.113813151474403e-06, 'stride': 100, 'n_eval_samples': 100}


In [4]:
# Category definitions (GICS level 1)

GICS_LEVEL_1 = {
    "Information Technology": [
        "AAPL", "ADBE", "ADI", "AMD", "AMAT", "CSCO", "FIS",
        "IBM", "INTC", "INTU", "LRCX", "MSFT", "MU",
        "NVDA", "ORCL", "QCOM", "TXN"
    ],
    "Health Care": [
        "ABBV", "ABT", "AMGN", "BDX", "BMY", "CVS", "DHR",
        "ELV", "GILD", "HUM", "ISRG", "JNJ",
        "LLY", "MRK", "PFE", "REGN", "SYK", "TMO", "VRTX", "ZTS"
    ],
    "Financials": [
        "AIG", "AXP", "BAC", "BLK", "C", "CB", "CI",
        "COF", "GS", "JPM", "MET", "MS",
        "PNC", "PGR", "SCHW", "USB", "WFC"
    ],
    "Consumer Discretionary": [
        "AMZN", "BKNG", "HD", "LOW", "MCD",
        "META", "NFLX", "NKE", "SBUX",
        "TGT", "TJX"
    ],
    "Consumer Staples": [
        "CL", "COST", "EL", "KMB", "KO",
        "MDLZ", "MO", "PEP", "PG", "PM", "WMT"
    ],
    "Industrials": [
        "BA", "CAT", "CSX", "DE", "EMR",
        "ETN", "FDX", "GD", "GE", "HON",
        "ITW", "LMT", "MMM", "NSC",
        "RTX", "UNP", "UPS", "WM"
    ],
    "Energy": ["COP", "CVX", "EOG", "OXY", "SLB", "XOM"],
    "Communication Services": ["CMCSA", "CRM", "GOOG", "GOOGL", "T", "TMUS", "VZ"],
    "Materials": ["APD", "LIN", "SHW"],
    "Real Estate": ["AMT", "SPG"],
    "Utilities": [],
}

def slugify(category: str) -> str:
    return (
        category.lower()
        .replace("&", "and")
        .replace("/", "_")
        .replace(" ", "_")
    )


In [5]:
# Locate checkpoints produced by the LoRA training notebook
# We support both layouts:
#   - running from repo root:        outputs/...
#   - running from notebooks folder: notebooks/outputs/...
# Choose the one that exists.
cand1 = Path(REPO_ROOT) / "notebooks" / "outputs"
cand2 = Path(REPO_ROOT) / "outputs"
OUTPUTS_BASE = cand1 if cand1.exists() else cand2

LORA_ROOT = OUTPUTS_BASE / "chronos2_sft_lora"
general_ckpt = LORA_ROOT / "general" / "finetuned-ckpt"

print("OUTPUTS_BASE:", OUTPUTS_BASE)
print("LORA_ROOT:", LORA_ROOT)

if not general_ckpt.exists():
    raise FileNotFoundError(f"LoRA general checkpoint not found: {general_ckpt}")

def category_ckpt(category: str) -> Path:
    return LORA_ROOT / slugify(category) / "finetuned-ckpt"


OUTPUTS_BASE: c:\Users\rosar\chronos_dnlp\notebooks\outputs
LORA_ROOT: c:\Users\rosar\chronos_dnlp\notebooks\outputs\chronos2_sft_lora


In [6]:
#  Metrics and window sampling (consistent with core/eval.py)

QUANTILES = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], dtype=np.float32)

def per_ticker_mql(y_true: np.ndarray, y_pred_quantiles: np.ndarray) -> np.ndarray:
    """Mean Quantile Loss (pinball) averaged over the 0.1..0.9 quantiles, per ticker.

    Important: some Chronos configs may output more than 9 quantiles. To stay consistent with our
    project evaluation (core/eval.py), we use the first 9 quantile columns (0.1..0.9) by index.
    """
    y_pred_9 = y_pred_quantiles[:, :len(QUANTILES)]  # (N, 9)
    errors = y_true[:, None] - y_pred_9              # (N, 9)
    q = QUANTILES[None, :]                           # (1, 9)
    pin = np.where(errors >= 0, q * errors, (q - 1.0) * errors)
    return pin.mean(axis=1)

def sample_start_indices(T: int, context_length: int, n_samples: int, seed: int = 42) -> np.ndarray:
    """Replicates the random window sampling used in core/eval.py::evaluate_model_on_test."""
    max_start = T - context_length - 1
    if max_start <= 0:
        raise ValueError(f"Series too short: T={T}, context={context_length}")
    np.random.seed(seed)
    n = min(n_samples, max_start)
    return np.random.choice(np.arange(max_start), size=n, replace=False)


In [7]:
# Evaluation + dump (per window × ticker)

def eval_dump_windows(
    pipeline: Chronos2Pipeline,
    df_test: pd.DataFrame,
    context_length: int,
    start_indices: np.ndarray,
    model_name: str,
    group_name: str,
    out_path: Path,
) -> pd.DataFrame:
    """Run a 1-step evaluation on sampled windows and save per-ticker results to Parquet."""
    data = df_test.values.astype(np.float32)  # shape (T, N)
    T, N = data.shape
    tickers = list(df_test.columns)
    dates = df_test.index

    rows = []
    for w_id, start in enumerate(tqdm(start_indices, desc=f"{model_name} | {group_name}", leave=False)):
        ctx = data[start:start+context_length, :].T        # (N, context)
        y_true = data[start+context_length, :]             # (N,)
        date = dates[start+context_length]

        forecast = pipeline.predict([{"target": ctx}], prediction_length=1)
        y_pred_q = forecast[0][:, :, 0].detach().cpu().numpy().astype(np.float32)  # (N, Q)
        y_med = y_pred_q[:, 4]  # median by index (consistent with our project setup)

        mae_t = np.abs(y_true - y_med)
        mse_t = (y_true - y_med) ** 2
        mql_t = per_ticker_mql(y_true, y_pred_q)

        # Store a row per ticker (window × ticker granularity)
        for i, tkr in enumerate(tickers):
            rows.append({
                "window_id": w_id,
                "start_idx": int(start),
                "date": date,
                "ticker": tkr,
                "group": group_name,
                "model": model_name,
                "context_length": int(context_length),
                "y_true": float(y_true[i]),
                "y_pred_q10": float(y_pred_q[i, 0]),
                "y_pred_q20": float(y_pred_q[i, 1]),
                "y_pred_q30": float(y_pred_q[i, 2]),
                "y_pred_q40": float(y_pred_q[i, 3]),
                "y_pred_q50": float(y_pred_q[i, 4]),
                "y_pred_q60": float(y_pred_q[i, 5]),
                "y_pred_q70": float(y_pred_q[i, 6]),
                "y_pred_q80": float(y_pred_q[i, 7]),
                "y_pred_q90": float(y_pred_q[i, 8]),
                "mae": float(mae_t[i]),
                "mse": float(mse_t[i]),
                "mql": float(mql_t[i]),
            })

    out_path.parent.mkdir(parents=True, exist_ok=True)
    df_out = pd.DataFrame(rows)
    df_out.to_parquet(out_path, index=False)
    print(f"Saved: {out_path}  (rows={len(df_out)})")
    return df_out


In [8]:
# Load models (baseline + LoRA general)

baseline = Chronos2Pipeline.from_pretrained("amazon/chronos-2", device_map=DEVICE, dtype=torch.float32)
lora_general = Chronos2Pipeline.from_pretrained(str(general_ckpt), device_map=DEVICE, dtype=torch.float32)

print("Baseline loaded.")
print("LoRA general loaded from:", general_ckpt)


Baseline loaded.
LoRA general loaded from: c:\Users\rosar\chronos_dnlp\notebooks\outputs\chronos2_sft_lora\general\finetuned-ckpt


In [9]:
# GLOBAL dumps

DUMPS_DIR = OUTPUTS_BASE / "eval_dumps" / "sft_lora"
DUMPS_DIR.mkdir(parents=True, exist_ok=True)

ctx_g = global_hp["context_length"]
n_g = global_hp["n_eval_samples"]
starts_g = sample_start_indices(len(df_eval_clean), ctx_g, n_g, seed=42)

global_baseline_path = DUMPS_DIR / "global__baseline.parquet"
global_lora_path = DUMPS_DIR / "global__lora_general.parquet"

_ = eval_dump_windows(baseline, df_eval_clean, ctx_g, starts_g, "baseline", "global", global_baseline_path)
_ = eval_dump_windows(lora_general, df_eval_clean, ctx_g, starts_g, "lora_general", "global", global_lora_path)


baseline | global:   0%|          | 0/100 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\global__baseline.parquet  (rows=11400)


lora_general | global:   0%|          | 0/100 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\global__lora_general.parquet  (rows=11400)


In [10]:
# Sanity check: aggregated metrics vs core/eval.py (global)

def agg_dump_metrics(parquet_path: Path) -> dict:
    d = pd.read_parquet(parquet_path)
    return {
        "mean_mql": float(d["mql"].mean()),
        "mean_mae": float(d["mae"].mean()),
        "mean_mse": float(d["mse"].mean()),
    }

print("Running core/eval.py evaluation (global) ...")
core_base = evaluate_model_on_test(baseline, df_eval_clean, context_length=ctx_g, n_samples=n_g)
core_lora = evaluate_model_on_test(lora_general, df_eval_clean, context_length=ctx_g, n_samples=n_g)

dump_base = agg_dump_metrics(global_baseline_path)
dump_lora = agg_dump_metrics(global_lora_path)

print("\nCORE baseline:", {k: core_base[k] for k in ["mean_quantile_loss","mean_mse","mean_mae"]})
print("DUMP baseline:", dump_base)

print("\nCORE lora   :", {k: core_lora[k] for k in ["mean_quantile_loss","mean_mse","mean_mae"]})
print("DUMP lora   :", dump_lora)

print("\nNote: core reports mean_quantile_loss; dump reports mean_mql (same concept, computed per ticker).")


Running core/eval.py evaluation (global) ...

CORE baseline: {'mean_quantile_loss': np.float32(0.0073292092), 'mean_mse': np.float32(0.0005467396), 'mean_mae': np.float32(0.016542953)}
DUMP baseline: {'mean_mql': 0.007329209622847358, 'mean_mae': 0.01654295417169985, 'mean_mse': 0.0005467396141380736}

CORE lora   : {'mean_quantile_loss': np.float32(0.006323756), 'mean_mse': np.float32(0.0004593847), 'mean_mae': np.float32(0.014547602)}
DUMP lora   : {'mean_mql': 0.006323756255968188, 'mean_mae': 0.014547601015714973, 'mean_mse': 0.0004593847257632555}

Note: core reports mean_quantile_loss; dump reports mean_mql (same concept, computed per ticker).


In [11]:
# CATEGORY dumps
# For each category, we dump:
#  - baseline (context = category context)
#  - LoRA general evaluated with global context (replicates the training notebook's choice)
#  - LoRA general evaluated with category context (fair paired comparison)
#  - LoRA category model (context = category context)
cats = []
for category, tickers in GICS_LEVEL_1.items():
    avail = [t for t in tickers if t in df_eval_clean.columns]
    if len(avail) > 0:
        cats.append((category, avail))

print("Categories with available tickers:", len(cats))

for category, tickers in cats:
    # Hyperparameters
    try:
        cat_hp = get_hparams(category)
    except KeyError:
        print(f"[{category}] skip: no tuned hyperparameters found in CSV.")
        continue

    df_cat = df_eval_clean[tickers]
    ctx_cat = cat_hp["context_length"]
    n_cat = cat_hp["n_eval_samples"]
    starts_cat = sample_start_indices(len(df_cat), ctx_cat, n_cat, seed=42)

    # Load category checkpoint if available
    ckpt = category_ckpt(category)
    if not ckpt.exists():
        print(f"[{category}] skip: checkpoint not found at {ckpt}")
        continue

    lora_cat = Chronos2Pipeline.from_pretrained(str(ckpt), device_map=DEVICE, dtype=torch.float32)

    slug = slugify(category)

    p_base = DUMPS_DIR / f"{slug}__baseline.parquet"
    p_cat  = DUMPS_DIR / f"{slug}__lora_category.parquet"
    p_gen_globalctx = DUMPS_DIR / f"{slug}__lora_general_ctx_global.parquet"
    p_gen_catctx    = DUMPS_DIR / f"{slug}__lora_general_ctx_cat.parquet"

    #  Baseline (category context)
    _ = eval_dump_windows(baseline, df_cat, ctx_cat, starts_cat, "baseline", slug, p_base)

    #  General LoRA (global context, as in training notebook category evaluation)
    ctx_global = global_hp["context_length"]
    starts_globalctx = sample_start_indices(len(df_cat), ctx_global, n_cat, seed=42)
    _ = eval_dump_windows(lora_general, df_cat, ctx_global, starts_globalctx, "lora_general", slug, p_gen_globalctx)

    #  General LoRA (category context, fair paired)
    _ = eval_dump_windows(lora_general, df_cat, ctx_cat, starts_cat, "lora_general", slug, p_gen_catctx)

    #  Category LoRA (category context)
    _ = eval_dump_windows(lora_cat, df_cat, ctx_cat, starts_cat, "lora_category", slug, p_cat)

    # Clean up GPU memory between categories
    del lora_cat
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("Done. Dumps saved to:", DUMPS_DIR)


Categories with available tickers: 10


baseline | information_technology:   0%|          | 0/100 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\information_technology__baseline.parquet  (rows=1700)


lora_general | information_technology:   0%|          | 0/100 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\information_technology__lora_general_ctx_global.parquet  (rows=1700)


...

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\real_estate__lora_general_ctx_global.parquet  (rows=300)


lora_general | real_estate:   0%|          | 0/150 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\real_estate__lora_general_ctx_cat.parquet  (rows=300)


lora_category | real_estate:   0%|          | 0/150 [00:00<?, ?it/s]

Saved: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora\real_estate__lora_category.parquet  (rows=300)
Done. Dumps saved to: c:\Users\rosar\chronos_dnlp\notebooks\outputs\eval_dumps\sft_lora
